## Recreating the Master Unit List

#### setup/imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import pudl
import pudl.constants as pc
import pudl.extract.ferc1
import sqlalchemy as sa
import logging
import sys
import copy
from copy import deepcopy

In [3]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline

In [4]:
import sys
sys.path.append("../") # go to parent dir
from plant_part_agg_eia import *
from plant_parts import plant_parts

In [232]:
logger = logging.getLogger()
logger.setLevel(logging.INFO)
handler = logging.StreamHandler(stream=sys.stdout)
formatter = logging.Formatter('%(message)s')
handler.setFormatter(formatter)
logger.handlers = [handler]

#### defining a table grabbing object

In [6]:
pudl_settings = pudl.workspace.setup.get_defaults()
pudl_engine = sa.create_engine(pudl_settings["pudl_db"])
pt = pudl.output.pudltabl.get_table_meta(pudl_engine)
start_date=None
end_date=None
freq='AS'

In [7]:
from plant_parts import plant_parts

In [8]:
table_compiler = CompileTables(pudl_engine, freq='AS', rolling=True)

In [27]:
self = CompilePlantParts(table_compiler, clobber=True)

In [472]:
plant_parts_df = self.generate_master_unit_list(plant_parts)

begin aggregation for: plant_prime_fuel
denormalize plant_prime_fuel
begin aggregation for: plant_technology
denormalize plant_technology
begin aggregation for: plant_prime_mover
denormalize plant_prime_mover
begin aggregation for: plant_gen
begin aggregation for: plant_unit
denormalize plant_unit
begin aggregation for: plant


In [473]:
cols = ['plant_id_eia', 'report_date', 'plant_part', 'peer_part','false_gran', 'generator_id',
       'unit_id_pudl', 'prime_mover_code', 'energy_source_code_1',
       'technology_description',]
print(len(plant_parts_df))
print(len(plant_parts_df[plant_parts_df['false_gran'] == True]))
plant_parts_df[(plant_parts_df['peer_part'].notnull())][cols]
#plant_parts_df['peer_part'].unique()


612178
524196


,plant_id_eia,report_date,plant_part,peer_part,false_gran,generator_id,unit_id_pudl,prime_mover_code,energy_source_code_1,technology_description
record_id_eia,,,,,,,,,,
2_wat_2011_plant_prime_fuel_total_195,2,2011-01-01,plant_prime_fuel,plant_prime_mover,True,NaN,NaN,HY,WAT,NaN
3_bit_2011_plant_prime_fuel_total_195,3,2011-01-01,plant_prime_fuel,plant_prime_mover,True,NaN,NaN,ST,BIT,NaN
3_ng_2011_plant_prime_fuel_total_195,3,2011-01-01,plant_prime_fuel,plant,True,NaN,NaN,NaN,NG,NaN
4_wat_2011_plant_prime_fuel_total_195,4,2011-01-01,plant_prime_fuel,plant_prime_mover,True,NaN,NaN,HY,WAT,NaN
7_bit_2011_plant_prime_fuel_total_195,7,2011-01-01,plant_prime_fuel,plant_prime_mover,True,NaN,NaN,ST,BIT,NaN
...,...,...,...,...,...,...,...,...,...,...
60464_1_2018_plant_unit_owned_62018,60464,2018-01-01,plant_unit,plant,True,NaN,1.0,NaN,NG,NaN
60464_1_2018_plant_unit_total_62018,60464,2018-01-01,plant_unit,plant,True,NaN,1.0,NaN,NG,NaN
60768_1_2018_plant_unit_total_16572,60768,2018-01-01,plant_unit,plant,True,NaN,1.0,NaN,NG,NaN


In [474]:
plant_parts_df[(plant_parts_df['false_gran'] == True) & 
               (plant_parts_df['peer_part'].isnull())
              ][cols]

,plant_id_eia,report_date,plant_part,peer_part,false_gran,generator_id,unit_id_pudl,prime_mover_code,energy_source_code_1,technology_description
record_id_eia,,,,,,,,,,


In [237]:
# 3) aggreate everything by each plant part
part_name = 'plant_technology'
plant_part = plant_parts[part_name]
logger.info(f'begin aggregation for: {part_name}')
id_cols = plant_part['id_cols']
ag_cols = plant_part['ag_cols']
wtavg_cols = plant_part['wtavg_cols']

if plant_part['denorm_table']:
    logger.info(f'denormalize {part_name}')
    df_in = self.denoramlize_table(self.plant_gen_df,
                                   id_cols,
                                   plant_part['denorm_table'],
                                   plant_part['denorm_cols'],
                                   )
else:
    df_in = self.plant_gen_df
thing = (
    self.table_compiler.agg_cols(
        id_cols=id_cols,
        ag_cols=ag_cols,
        wtavg_cols=wtavg_cols,
        df_in=df_in).
    pipe(self.add_install_year, id_cols,
         plant_part['install_table']).
    assign(plant_part=part_name))

begin aggregation for: plant_technology
denormalize plant_technology


In [418]:
thing = part_df
plant_id = 2322
part_name = 'plant_technology'
#part_df = plant_parts_df[plant_parts_df['plant_part'] == part_name]
id_cols = plant_parts[part_name]['id_cols']

#false_gran = plant_parts[part_name]['false_grans']
false_gran = ['plant_gen', 'plant_prime_mover']
if false_gran:
    if 'false_gran' not in part_df.columns:
        part_df['false_gran'] = np.nan
    for peer_part in false_gran:
        id_cols_peer = plant_parts[peer_part]['id_cols']
        logger.debug(f'labeling false granularities from {peer_part}')
        # prepare a df for counting
        if plant_parts[peer_part]['denorm_table']:
            logger.debug('grabbing denorm_table')
            part_peer_df = self.table_compiler.grab_the_table(
                plant_parts[peer_part]['denorm_table'])
        else:
            part_peer_df = part_df
        if (not set(id_cols_peer).issubset(list(part_peer_df.columns))
                and peer_part == 'plant_gen'):
            logger.debug('using the plant_gen_df for the part_peer_df')
            part_peer_df = self.plant_gen_df
        if not set(id_cols).issubset(list(part_peer_df.columns)):
            logger.debug(f'merging the id_cols for into the {peer_part} table')
            #TODO: fix this brooorken merge
            part_peer_df = part_peer_df.merge(
                self.table_compiler.grab_the_table(
                plant_parts[part_name]['denorm_table'])[id_cols + ['generator_id']].drop_duplicates())
        part_out_df = self._find_false_grans(
            part_df, part_peer_df, id_cols, id_cols_peer, peer_part, part_name)

In [ ]:
part

In [420]:
part_out_df[part_out_df['plant_id_eia'] == plant_id]#[['plant_id_eia','technology_description', 'count']]

,plant_id_eia,technology_description,count,report_date,plant_part,generator_id,unit_id_pudl,prime_mover_code,energy_source_code_1,utility_id_eia,...,net_generation_mwh,ownership,plant_id_pudl,plant_name_eia,total_fuel_cost,total_mmbtu,utility_id_pudl,utility_name_eia,report_year,plant_id_report_year
9677,2322,Natural Gas Fired Combined Cycle,2,2014-01-01,plant_technology,NaN,NaN,NaN,NG,13407,...,389683.0,total,116,Clark (NVE),0.000000e+00,0.000,204,Nevada Power Co,2014,116_2014
9678,2322,Natural Gas Fired Combined Cycle,2,2015-01-01,plant_technology,NaN,NaN,NaN,NG,13407,...,602429.0,total,116,Clark (NVE),1.914872e+07,5639462.778,204,Nevada Power Co,2015,116_2015
9679,2322,Natural Gas Fired Combined Cycle,2,2016-01-01,plant_technology,NaN,NaN,NaN,NG,13407,...,647931.0,total,116,Clark (NVE),1.869425e+07,5932080.803,204,Nevada Power Co,2016,116_2016
9680,2322,Natural Gas Fired Combined Cycle,2,2017-01-01,plant_technology,NaN,NaN,NaN,NG,13407,...,486873.0,total,116,Clark (NVE),1.648939e+07,4721643.212,204,Nevada Power Co,2017,116_2017
9681,2322,Natural Gas Fired Combined Cycle,2,2018-01-01,plant_technology,NaN,NaN,NaN,NG,13407,...,535881.0,total,116,Clark (NVE),1.375768e+07,5147474.740,204,Nevada Power Co,2018,116_2018
9682,2322,Natural Gas Fired Combustion Turbine,1,2014-01-01,plant_technology,NaN,NaN,GT,NG,13407,...,0.0,total,116,Clark (NVE),0.000000e+00,0.000,204,Nevada Power Co,2014,116_2014
9683,2322,Natural Gas Fired Combustion Turbine,1,2015-01-01,plant_technology,NaN,NaN,GT,NG,13407,...,0.0,total,116,Clark (NVE),0.000000e+00,0.000,204,Nevada Power Co,2015,116_2015
9684,2322,Natural Gas Fired Combustion Turbine,1,2016-01-01,plant_technology,NaN,NaN,GT,NG,13407,...,0.0,total,116,Clark (NVE),0.000000e+00,0.000,204,Nevada Power Co,2016,116_2016
9685,2322,Natural Gas Fired Combustion Turbine,1,2017-01-01,plant_technology,NaN,NaN,GT,NG,13407,...,0.0,total,116,Clark (NVE),0.000000e+00,0.000,204,Nevada Power Co,2017,116_2017
9686,2322,Natural Gas Fired Combustion Turbine,1,2018-01-01,plant_technology,NaN,NaN,GT,NG,13407,...,0.0,total,116,Clark (NVE),0.000000e+00,0.000,204,Nevada Power Co,2018,116_2018


In [416]:
peer_pre = (part_peer_df.drop_duplicates(
                subset=set(id_cols_peer + id_cols)).
            pipe(pudl.helpers.count_records, id_cols, 'count').
            merge(part_df.drop(columns=['count']), how='right')
           )
count = pudl.helpers.count_records(
            peer_pre, id_cols, 'count')
peer_count = count.merge(
        # we need to get the count of the peer parts ids
        part_peer_df, how='left')#[id_cols + ['count']].drop_duplicates()

In [398]:
peer_pre = part_peer_df.drop_duplicates(
                subset=set(id_cols_peer + id_cols))
count = pudl.helpers.count_records(
            peer_pre, id_cols, 'count')
peer_count = count.merge(
        # we need to get the count of the peer parts ids
        part_peer_df, how='left')#[id_cols + ['count']].drop_duplicates()

In [417]:
peer_pre[peer_pre['plant_id_eia']==plant_id][id_cols + ['count']]

,plant_id_eia,technology_description,count
9677,2322,Natural Gas Fired Combined Cycle,2
9678,2322,Natural Gas Fired Combined Cycle,2
9679,2322,Natural Gas Fired Combined Cycle,2
9680,2322,Natural Gas Fired Combined Cycle,2
9681,2322,Natural Gas Fired Combined Cycle,2
9682,2322,Natural Gas Fired Combustion Turbine,1
9683,2322,Natural Gas Fired Combustion Turbine,1
9684,2322,Natural Gas Fired Combustion Turbine,1
9685,2322,Natural Gas Fired Combustion Turbine,1
9686,2322,Natural Gas Fired Combustion Turbine,1


In [378]:
#part_peer_df[part_peer_df['plant_id_eia']==plant_id][['plant_id_eia','prime_mover_code','count']]

In [379]:
part_count[part_count['plant_id_eia'] == plant_id][id_cols + ['count']]

,plant_id_eia,technology_description,count
3713,2322,Natural Gas Fired Combined Cycle,3.0
3714,2322,Natural Gas Fired Combustion Turbine,3.0
3715,2322,Natural Gas Steam Turbine,3.0
15552,2322,Natural Gas Fired Combined Cycle,3.0
15553,2322,Natural Gas Fired Combustion Turbine,3.0
15554,2322,Natural Gas Steam Turbine,3.0
27836,2322,Natural Gas Fired Combined Cycle,3.0
27837,2322,Natural Gas Fired Combustion Turbine,3.0
27838,2322,Natural Gas Steam Turbine,3.0
40943,2322,Natural Gas Fired Combined Cycle,3.0


In [229]:
parts_parts_df['false_gran_test'] = parts_parts_df.filter(like='false_gran_', axis=1).any(axis=1)

In [227]:
part_truth['truth?']= part_truth['false_gran_test'] == part_truth['false_gran']

In [228]:
part_truth[part_truth['truth?'] == False]

,plant_id_eia,report_date,plant_part,generator_id,unit_id_pudl,prime_mover_code,energy_source_code_1,technology_description,false_gran,false_gran_plant_gen_plant_prime_fuel,...,false_gran_plant_plant_unit,false_gran_plant_prime_mover_plant_prime_fuel,false_gran_plant_prime_mover_plant_technology,false_gran_plant_technology_plant_prime_fuel,false_gran_plant_unit_plant_gen,false_gran_plant_unit_plant_prime_fuel,false_gran_plant_unit_plant_prime_mover,false_gran_plant_unit_plant_technology,false_gran_test,truth?
record_id_eia,,,,,,,,,,,,,,,,,,,,,


In [ ]:
compiled_plant_parts = {}
for part in plant_parts.keys():
    compiled_plant_parts[part] = plant_parts_df[(plant_parts_df['plant_part'] == part) & 
                                                (plant_parts_df[plant_parts_df.false_gran != True])]

In [ ]:
def plot_plant_vs_agg(compiled_plant_parts, field, xy_limits, scale):
    """
    Make plots to compare FERC & EIA reported values for Coal & Gas plants.
    
    For each of the fields specified in fields_to_plot, create a pair of plots,
    one for 'gas' and one for 'coal' in the same frame, showing the EIA quantity
    vs. the FERC quantity in a scatter plot.
    """
    for plant_gran, df in compiled_plant_parts.items():
        if plant_gran != 'plant':
            field_plant = field+'_plant'
            field_gran = field+'_'+plant_gran
            try:
                merge_df = compiled_plant_parts['plant'].merge(df, on=['plant_id_eia', 
                                                                       'report_date',
                                                                       'utility_id_eia',
                                                                       'fraction_owned'
                                                                      ], suffixes=('_plant','_' + plant_gran))
                # this is for the try
                merge_df[field_gran]
                #merge_df = merge_df[merge_df['plant_id_eia'] == 3]
                fig, (ax) = plt.subplots(ncols=1, nrows=1, figsize=(5, 5))
                ax.scatter(merge_df[field_plant],
                           merge_df[field_gran],
                           color='black', alpha='0.1', label=field)
                ax.set_ylim(xy_limits[field][0],xy_limits[field][1])
                ax.set_xlim(xy_limits[field][0],xy_limits[field][1])
                ax.set_xscale(scale)
                ax.set_yscale(scale)
                ax.set_ylabel(f'{plant_gran} {field}')
                ax.set_xlabel(f'Plant {field}')
                ax.set_title(f"Plant vs {plant_gran}: {field}")
            except KeyError:
                pass

In [ ]:
fields_to_plot = [
    # Simple Quantities
    'capacity_mw',
    #'opex_fuel',
    'net_generation_mwh',
    'total_mmbtu',
    # Derived values
    #'capacity_factor',
    'heat_rate_mmbtu_mwh',
    'fuel_cost_per_mwh',
    'fuel_cost_per_mmbtu',
    'total_fuel_cost'
]

xy_limits = {
    # Simple Quantities
    'capacity_mw': (1e0, 1e4),
    'net_generation_mwh': (1e3,1e8),
    'total_mmbtu': (1e4,1e9),
    # Derived values
    'capacity_factor': (0,1.0),
    'heat_rate_mmbtu_mwh': (6,16),
    'fuel_cost_per_mwh': (10,80),
    'fuel_cost_per_mmbtu': (1e0,1e1),
    'total_fuel_cost': (1e7,1e9)
}

for field in fields_to_plot:
    plot_plant_vs_agg(compiled_plant_parts,field, xy_limits, scale="log")

In [ ]:
def plot_gens_vs(compiled_plant_parts,part_name, data_col, weight_col, x_range):
    gen_df = compiled_plant_parts['plant_gen'][compiled_plant_parts['plant_gen'][data_col] != 0]
    part_df = compiled_plant_parts[part_name][compiled_plant_parts[part_name][data_col] != 0]
    if weight_col:
        weights_gen = gen_df[weight_col]
        weights_part = part_df[weight_col]
    else:
        weights_gen = None
        weights_part = None

    plt.hist(gen_df[data_col], 
             weights=weights_gen,
             range=x_range,
             bins=100,
             color="purple", #alpha=test_alpha,
             label="Generators")

    plt.hist(part_df[data_col], 
             weights=weights_part,
             range=x_range,
             bins=100,
             color="black",
             label=f'{part_name}')

    plt.title(f'Gens vs. {part_name}: {data_col}')
    plt.xlabel(data_col)
    plt.ylabel(None)
    plt.legend()
    plt.show()

In [ ]:
x_ranges = {
    'capacity_mw' : (0,400),
    'net_generation_mwh': (0, 2500000),
    'fuel_cost_per_mmbtu': (0, 5),
    'fuel_cost_per_mwh': (0, 100),
    'total_fuel_cost': (0,200000000)
}

In [ ]:
for part_name in compiled_plant_parts.keys():
    data_col =  'net_generation_mwh'
    weight_col = 'capacity_mw'
    plot_gens_vs(compiled_plant_parts,
                 part_name=part_name,
                 data_col=data_col, 
                 weight_col=weight_col,
                 x_range=x_ranges[data_col])

In [ ]:
for part_name in compiled_plant_parts.keys():
    data_col =  'total_fuel_cost'
    weight_col = 'capacity_mw'
    plot_gens_vs(compiled_plant_parts,
                 part_name=part_name,
                 data_col=data_col, 
                 weight_col=weight_col,
                 x_range=x_ranges[data_col])

In [ ]:
for part_name in compiled_plant_parts.keys():
    data_col =  'fuel_cost_per_mwh'
    weight_col = 'capacity_mw'
    plot_gens_vs(compiled_plant_parts,
                 part_name=part_name,
                 data_col=data_col, 
                 weight_col=weight_col,
                 x_range=x_ranges[data_col])

In [ ]:
for part_name in compiled_plant_parts.keys():
    data_col =  'fuel_cost_per_mmbtu'
    weight_col = 'capacity_mw'
    plot_gens_vs(compiled_plant_parts,
                 part_name=part_name,
                 data_col=data_col, 
                 weight_col=weight_col,
                 x_range=x_ranges[data_col])

## Playing with the compiled outputs 

In [ ]:
# various tables to poke at for debuging
generat923 = table_complier.grab_the_table('generation_eia923')
gens860 = table_complier.grab_the_table('generators_eia860')
bga860 = table_complier.grab_the_table('boiler_generator_assn_eia860')
own860 = table_complier.grab_the_table('ownership_eia860')
plant = compiled_plant_parts['plant']
plant_gen = compiled_plant_parts['plant_gen']
plant_unit = compiled_plant_parts['plant_unit']
plant_fuel = compiled_plant_parts['plant_prime_fuel']
plant_tech = compiled_plant_parts['plant_technology']
plant_pm = compiled_plant_parts['plant_prime_mover']

In [ ]:
# this can be used to see if the aggregations generally worked properly
plant_gen_agged = plant_gen.groupby(by=['utility_id_eia','plant_id_eia','fraction_owned','report_date']).sum().reset_index()
merge = plant.merge(plant_gen_agged,
                    on=['utility_id_eia','plant_id_eia','fraction_owned','report_date'],
                    suffixes=('_plant','_gen')).drop(columns=['fuel_cost_per_mmbtu_plant','fuel_cost_per_mmbtu_gen',
                                                              'heat_rate_mmbtu_mwh_plant', 'heat_rate_mmbtu_mwh_gen',
                                                              'fuel_cost_per_mwh_plant','fuel_cost_per_mwh_gen',
                                                             ])

In [ ]:
count_df = pd.DataFrame(index=list(compiled_plant_parts['plant'].columns))
for k,cpp_df in compiled_plant_parts.items():
    count_df = count_df.merge(
        pd.DataFrame(cpp_df[cpp_df['report_date'].dt.year == 2018]
                     .count(), columns=[k]),
        right_index=True, left_index=True)
count_df = count_df.merge(pd.DataFrame(count_df.sum(axis=1),columns=['total']),
                          right_index=True, left_index=True)

In [ ]:
count_df

In [431]:
table_compiler._dfs.keys()

dict_keys(['generation_fuel_eia923', 'fuel_receipts_costs_eia923', 'generators_eia860', 'boiler_generator_assn_eia860', 'generation_eia923', 'ownership_eia860', 'generators_entity_eia', 'utilities_eia', 'plants_eia', 'energy_source_eia923', 'fuel_cost', 'mcoe', 'plants_steam_ferc1'])

In [434]:
mcoe = table_compiler._dfs['mcoe']
gen_ent = table_compiler._dfs['generators_entity_eia']
plants_eia = table_compiler._dfs['plants_eia']

In [429]:
mcoe[mcoe['state'] == 'DC']

,report_date,plant_id_eia,plant_id_pudl,unit_id_pudl,generator_id,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,balancing_authority_code,...,technology_description,time_cold_shutdown_full_load_code,total_fuel_cost,total_mmbtu,turbines_inverters_hydrokinetics,turbines_num,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_mw,winter_estimated_capability_mw


,plant_id_eia,plant_name_eia,plant_id_pudl
0,2,Bankhead Dam,853
1,3,Barry,32
2,4,Walter Bouldin Dam,852
3,7,Gadsden,206
4,8,Gorgas,229
...,...,...,...
10440,62634,babcock preserve,9929
10441,70454,hartwell energy facility,4129
10442,99999,State-Fuel Level Increment,7604
10443,880016,bayway refinery,4915


In [435]:
plants_eia[plants_eia['state'] == 'DC']

KeyError: 'state'

In [430]:
mcoe[mcoe['state'] == 'DC']

,report_date,plant_id_eia,plant_id_pudl,unit_id_pudl,generator_id,plant_name_eia,utility_id_eia,utility_id_pudl,utility_name_eia,balancing_authority_code,...,technology_description,time_cold_shutdown_full_load_code,total_fuel_cost,total_mmbtu,turbines_inverters_hydrokinetics,turbines_num,uprate_derate_completed_date,uprate_derate_during_year,winter_capacity_mw,winter_estimated_capability_mw
0,2011-01-01,3,32,1,1,Barry,195,18,Alabama Power Co,SOCO,...,None,None,1.531524e+07,3.565315e+06,NaN,NaN,NaT,False,138.0,NaN
7,2012-01-01,3,32,1,1,Barry,195,18,Alabama Power Co,SOCO,...,None,None,8.185375e+06,1.743140e+06,NaN,NaN,NaT,False,138.0,NaN
54,2017-01-01,3,32,1,1,Barry,195,18,Alabama Power Co,SOCO,...,Natural Gas Steam Turbine,OVER,6.577052e+05,1.966533e+05,NaN,NaN,NaT,False,55.0,NaN
1,2011-01-01,3,32,2,2,Barry,195,18,Alabama Power Co,SOCO,...,None,None,9.821807e+06,2.286471e+06,NaN,NaN,NaT,False,137.0,NaN
8,2012-01-01,3,32,2,2,Barry,195,18,Alabama Power Co,SOCO,...,None,None,7.015256e+06,1.493954e+06,NaN,NaN,NaT,False,137.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25353,2016-01-01,59784,236,1,CTG6,Gila River Power Block 3,24211,330,Tucson Electric Power Co,GRMA,...,Natural Gas Fired Combined Cycle,1H,1.711166e+07,5.846617e+06,NaN,NaN,NaT,False,163.0,NaN
25356,2017-01-01,59784,236,1,CTG6,Gila River Power Block 3,24211,330,Tucson Electric Power Co,GRMA,...,Natural Gas Fired Combined Cycle,1H,1.659963e+07,4.862373e+06,NaN,NaN,NaT,False,163.0,NaN
25351,2015-01-01,59784,236,1,ST11,Gila River Power Block 3,24211,330,Tucson Electric Power Co,GRMA,...,Natural Gas Fired Combined Cycle,12H,2.026677e+07,6.525114e+06,NaN,NaN,NaT,False,227.0,NaN
25354,2016-01-01,59784,236,1,ST11,Gila River Power Block 3,24211,330,Tucson Electric Power Co,GRMA,...,Natural Gas Fired Combined Cycle,12H,2.099235e+07,7.172553e+06,NaN,NaN,NaT,False,227.0,NaN
